# EvoGFuzz: An Evolutionary Approach to Grammar-Based Fuzzing

**EvoGFuzz** stands for *evolutionary grammar-based fuzzing*. This approach leverages evolutionary optimization techniques to systematically explore the space of a program's potential inputs, with a particular emphasis on identifying inputs that could lead to exceptional behavior. With a user-defined objective, EvoGFuzz can adapt and refine the input generation strategy over time, making it a powerful tool for uncovering software defects and vulnerabilities.

Efficient detection of defects and vulnerabilities hinges on the ability to automatically generate program inputs that are both valid and diverse. One common strategy is to use grammars, which provide structured and syntactically correct inputs. This approach leads to the concept of grammar-based fuzzing, where fuzzing strategies are guided by the rules defined within the grammar.

A further enhancement to this concept is probabilistic grammar-based fuzzing, where competing grammar rules are associated with probabilities that guide their application. By carefully assigning and optimizing these probabilities, we gain considerable control over the nature of the generated inputs. This enables us to direct the fuzzing process towards specific areas of interest—for example, those functions that are deemed critical, have a higher propensity for failures, or have undergone recent modifications. 

In essence, EvoGFuzz represents a potent blend of evolutionary optimization and probabilistic grammar-based fuzzing, poised to reveal hidden defects and vulnerabilities in a targeted and efficient manner.

## Fuzzing a Program

Our program under investigation is `The Calculator`. This program acts as a typical calculator, capable of evaluating not just arithmetic expressions but also trigonometric functions, such as sine, cosine, and tangent. Furthermore, it also supports the calculation of the square root of a given number.

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions 
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

**Side Note:** In the `calculator`, we use Python's `eval` function, which takes a string and evaluates it as a Python expression. We provide a dictionary as the second argument to eval, mapping names to corresponding mathematical functions. This enables us to use the function names directly within the input string. 

In [2]:
# Evaluating the cosine of 2π
print(calculator('cos(6*3.141)'))

0.999993677717667


In [3]:
# Calculating the square root of 36
print(calculator('sqrt(6*6)'))

6.0


Each of these calls to the calculator will evaluate the provided string as a mathematical expression, and print the result.

Now, to find new defects, we need to introduce an oracle that tells us if the error that is triggered is something we expect or a new/unkonwn defect. The `OracleResult` is an enum with two possible values, `NO_BUG` and `BUG`. `NO_BUG` donates a passing test case and `BUG` a failing one.

We import the `OracleResult` enumerated type from the `evogfuzz` library. This is used in the oracle function to indicate the outcome of executing the 'calculator' function with a given input.

In [4]:
from evogfuzz.oracle import OracleResult

This is a function called **oracle**, which acts as an intermediary to handle and classify exceptions produced by the calculator function when given a certain input.

In [5]:
# Make sure you use the OracleResult from the evogfuzz library
from evogfuzz.oracle import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function. The oracle function is used in the context of fuzz testing.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.NO_BUG: Returned if the calculator function executes without any exception or only with CalculatorSyntaxError
            - OracleResult.BUG: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.BUG
    
    return OracleResult.NO_BUG

This **oracle** function is used in the context of fuzzing to determine the impact of various inputs on the program under test (in our case the _calculator_). When the calculator function behaves as expected (i.e., no exceptions occur), the **oracle** function returns `OracleResult.NO_BUG`. However, when the `calculator` function raises an unexpected exception, the **oracle** interprets this as a potential bug in the `calculator` and returns `OracleResult.BUG`.

We can see this in action by testing a few initial inputs:

In [6]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)']

for inp in initial_inputs:
    print(inp.ljust(20), oracle(inp))

sqrt(1)              NO_BUG
cos(912)             NO_BUG
tan(4)               NO_BUG


The following code represents a simple context-free grammar for our calculator function. This grammar encompasses all the potential valid inputs to the calculator, which include mathematical expressions involving square roots, trigonometric functions, and integer and decimal numbers:

In [7]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar

CALCGRAMMAR: Grammar = {
    "<start>":
        ["<function>(<term>)"],

    "<function>":
        ["sqrt", "tan", "cos", "sin"],
    
    "<term>": ["-<value>", "<value>"], 
    
    "<value>":
        ["<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
}
    
grammar_alhazen: Grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": [str(num) for num in range(0, 10)],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"],
}
    
assert is_valid_grammar(grammar_alhazen)

The defined grammar CALCGRAMMAR provides a structured blueprint for creating various inputs for our fuzz testing. Each rule in this grammar reflects a possible valid input that our calculator function can handle. By fuzzing based on this grammar, we can systematically explore the space of valid inputs to the calculator function.

### Leveraging EvoGFuzz to Unearth New Defects

We apply our `EvoGFuzz` class to carry out fuzz testing using evolutionary grammar-based fuzzing. This is aimed at uncovering potential defects in our 'calculator' function.

To initialize our EvoGFuzz instance, we require a grammar (in our case, `CALCGRAMMAR`), an oracle function, an initial set of inputs, a fitness function, and the number of iterations to be performed in the fuzzing process.

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [15]:
from evogfuzz.evogfuzz_class import EvoGFuzz
from evogfuzz.helper import Tournament_Selection_Mode

epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_LEVENSHTEIN
)

Upon creating the `EvoGFuzz` instance, we can execute the fuzzing process. The `.fuzz()` method runs the fuzzing iterations, evolving the inputs based on our fitness function, and returns a collection of inputs that lead to exceptions in the 'calculator' function.

In [16]:
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

4 ['sqrt(22)', 'sqrt(11)', 'sqrt(44)', 'tan(911)', 'tan(411)', 'cos(244)']
6 ['cos(1141914)', 'cos(11)', 'tan(99)', 'tan(249)', 'sqrt(12)', 'cos(92922)', 'sqrt(94)', 'sqrt(111)', 'cos(421)', 'tan(491)']
5 ['sqrt(112)', 'tan(4211)', 'sqrt(119)', 'sqrt(292)', 'sqrt(411)', 'sqrt(424)', 'sqrt(991)']
1 ['tan(21)', 'tan(14)', 'tan(24)', 'cos(41)', 'tan(19)', 'cos(29)', 'tan(41)', 'tan(29)', 'tan(92)']
3 ['tan(1)', 'tan(2)', 'cos(9)', 'cos(4)', 'tan(9)', 'cos(1)', 'cos(2)', 'tan(4)']
2 ['cos(91)', 'sqrt(9)', 'sqrt(4)', 'sqrt(1)', 'sqrt(2)', 'cos(19)']
4 6
6 10
5 7
1 9
3 8
2 6
1 ['cos(-1)', 'sqrt(4)', 'sqrt(9)', 'tan(49)', 'tan(94)', 'sqrt(1)', 'cos(-9)', 'tan(-1)', 'tan(-4)', 'tan(91)', 'cos(94)']
4 ['tan(44)', 'tan(99)', 'tan(11)']
2 ['tan(1)', 'cos(9)', 'cos(4)', 'tan(9)', 'cos(1)', 'tan(4)']
3 ['tan(-91)', 'sqrt(91)', 'sqrt(49)', 'sqrt(19)', 'cos(419)']
5 ['tan(-11)', 'sqrt(919)', 'sqrt(11)', 'sqrt(1949)', 'cos(4919)', 'tan(144)', 'cos(991)', 'sqrt(9944)', 'tan(9914)', 'tan(111)']
1 11
4 3

Lastly, we can examine the inputs that resulted in exceptions. This output can provide valuable insight into potential weaknesses in the 'calculator' function that need to be addressed.

In [ ]:
# print only the first 20 bug-triggering inputs
for inp in list(found_exception_inputs)[:20]:
    print(str(inp))

In [14]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_JARO
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

7 ['sqrt(1992)', 'tan(9141)', 'cos(94992)', 'sqrt(9144)', 'sqrt(911)', 'tan(19294)']
6 ['tan(11111)', 'sqrt(99)', 'tan(99)', 'sqrt(44)', 'tan(44)', 'tan(22)', 'cos(9191)', 'sqrt(214)']
1 ['cos(91)', 'cos(92)', 'tan(19)', 'cos(49)', 'cos(29)', 'cos(19)', 'tan(91)', 'cos(94)']
2 ['sqrt(4)', 'sqrt(9)', 'tan(14)', 'sqrt(1)', 'sqrt(2)', 'cos(24)', 'tan(12)', 'tan(41)', 'tan(29)', 'cos(21)']
5 ['sqrt(111)', 'sqrt(11299)', 'cos(111)', 'tan(111)']
3 ['tan(1)', 'cos(9)', 'cos(4)', 'cos(2)', 'tan(9)', 'cos(1)', 'tan(2)', 'tan(4)']
4 ['sqrt(14)', 'sqrt(24)', 'tan(421)', 'cos(294)', 'sqrt(91)', 'sqrt(94)', 'sqrt(42)', 'sqrt(19)']
7 6
6 8
1 8
2 10
5 4
3 8
4 8
4 ['sin(-1)', 'cos(-8)', 'cos(-1)', 'cos(76)', 'cos(-4)', 'sin(-7)', 'sin(-3)', 'cos(24)', 'cos(-5)', 'cos(-2)', 'sin(18)', 'cos(79)', 'tan(38)', 'tan(53)', 'tan(-5)']
7 ['cos(-3267726.43)', 'tan(8)', 'cos(-957)', 'tan(69236466)', 'cos(919.5)', 'cos(242.5)', 'tan(-8.365118)', 'cos(5.53451)', 'cos(238312.44)', 'cos(6)', 'cos(7)', 'cos(1)', 'cos

3 ['sqrt(-75)', 'sqrt(981)', 'sqrt(-71)', 'sqrt(-79)', 'sqrt(278)', 'sqrt(-48)', 'sqrt(-67)', 'sqrt(-69)', 'sqrt(715)', 'sqrt(247)', 'sqrt(915)', 'sqrt(451)', 'sqrt(-12)', 'sqrt(-27)', 'sqrt(-46)']
8 ['sqrt(177)', 'sqrt(-7589)', 'sqrt(89714)', 'sqrt(-9916)', 'sqrt(-88)', 'sqrt(727)', 'sqrt(557)', 'sqrt(-9198)', 'sqrt(797)']
7 ['sqrt(57957)', 'sqrt(99887)', 'sqrt(4593474)', 'sqrt(1782191)', 'sqrt(79678569)', 'sqrt(68558)', 'sqrt(-19771)']
9 ['sqrt(9995)', 'sqrt(-192817)', 'sqrt(-37977577779)', 'sqrt(585)', 'sqrt(-975711)', 'sqrt(-67288781)', 'sqrt(73888)', 'sqrt(97757)', 'sqrt(8797)']
2 ['sqrt(7)', 'sqrt(9)', 'sqrt(4)', 'sqrt(6)', 'sqrt(8)', 'sqrt(5)']
5 ['sqrt(-968)', 'sqrt(2679)', 'sqrt(-295)', 'sqrt(9865)', 'sqrt(-159)', 'sqrt(-695)']
1 ['sqrt(-6)', 'sqrt(-3)', 'sqrt(-9)', 'sqrt(65)', 'sqrt(-4)', 'sqrt(25)', 'sqrt(-7)', 'sqrt(-5)', 'sqrt(96)', 'sqrt(78)', 'sqrt(72)', 'sqrt(89)', 'sqrt(58)', 'sqrt(17)', 'sqrt(52)', 'sqrt(79)', 'sqrt(67)', 'sqrt(68)', 'sqrt(-8)', 'sqrt(97)']
6 ['sqrt(-

In [19]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
    tournament_selection_mode = Tournament_Selection_Mode.HIERARCHICAL_FEATURE_COS
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

2 ['tan(49)', 'tan(492)', 'tan(99)', 'tan(944)', 'tan(44)', 'tan(2944)', 'tan(922)', 'tan(42)', 'tan(9)', 'tan(2)', 'tan(4)']
1 ['tan(2912)', 'tan(4291)', 'tan(1)', 'tan(14)', 'tan(4192)', 'tan(91)', 'tan(19)', 'tan(41919)', 'tan(41)', 'tan(219)']
4 ['cos(91)', 'cos(41)', 'cos(1219)', 'cos(9)', 'cos(29)', 'cos(4)', 'cos(21)', 'cos(129)', 'cos(2)', 'cos(14)', 'cos(94)']
5 ['sqrt(42294)', 'sqrt(9224)', 'sqrt(914)', 'sqrt(94)', 'sqrt(92941)']
6 ['sqrt(41)', 'sqrt(4)', 'sqrt(9)', 'sqrt(14)', 'sqrt(1)', 'sqrt(2)', 'sqrt(11)', 'sqrt(91)', 'sqrt(19)']
3 ['cos(1)', 'cos(11)']
2 11
1 10
4 11
5 5
6 9
3 2
1 ['tan(1949239)', 'tan(49492111)', 'tan(195199)', 'tan(9191)', 'tan(21999)', 'tan(9151945)', 'tan(99329)']
5 ['tan(39)', 'tan(49)', 'tan(99)', 'tan(19)', 'tan(59)', 'tan(395)', 'tan(392)', 'tan(192394)', 'tan(9)', 'tan(1)', 'tan(192)', 'tan(29563)', 'tan(41)', 'tan(13)', 'tan(919445)', 'tan(28396432)', 'tan(12193)', 'tan(91)', 'tan(9134)', 'tan(21592)', 'tan(97)', 'tan(61)', 'tan(91421)', 'tan(

In [26]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    iterations=10,
)
found_exception_inputs = epp.fuzz()
print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

EvoGFuzz found 230 bug-triggering inputs!


This process illustrates the power of evolutionary grammar-based fuzzing in identifying new defects within our system. By applying evolutionary algorithms to our fuzzing strategy, we can guide the search towards more defect-prone regions of the input space.

#### Analyzing and Sorting All Generated Inputs by Fitness

After the fuzzing process, you may want to examine all the generated inputs. These can be accessed using the `get_all_inputs()` method. Additionally, we can sort these inputs based on their fitness scores to gain insights into which inputs performed best according to our fitness function.

In [27]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Now, let's print out these sorted inputs along with their respective fitness scores. Inputs with higher fitness scores will be displayed first, as these are the ones our evolutionary process deemed more likely to uncover potential defects.

In [28]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-276.2513235)                       fitness: 1
sqrt(-3.32)                              fitness: 1
sqrt(-223)                               fitness: 1
sqrt(-4316.2)                            fitness: 1
sqrt(-3.6598428597)                      fitness: 1
sqrt(-3.7691)                            fitness: 1
sqrt(-8.557)                             fitness: 1
sqrt(-7232535.2)                         fitness: 1
sqrt(-3266662322.225271)                 fitness: 1
sqrt(-12.433)                            fitness: 1
sqrt(-858.18)                            fitness: 1
sqrt(-858)                               fitness: 1
sqrt(-5.66)                              fitness: 1
sqrt(-77.5438)                           fitness: 1
sqrt(-7.3)                               fitness: 1
sqrt(-8.85426988736)                     fitness: 1
sqrt(-8.772286)                          fitness: 1
sqrt(-57777)                             fitness: 1
sqrt(-5633)                              fitness: 1
sqrt(-636421

This output provides an overview of the evolved inputs and their effectiveness in revealing potential defects, as gauged by our fitness function. It is a valuable resource for understanding the behavior of our program under various inputs and the effectiveness of our evolutionary grammar-based fuzzing approach.

### Incorporating Custom Fitness Functions

The fitness function plays a crucial role in guiding the evolution process of our fuzzing inputs. A well-crafted fitness function can effectively direct the search towards the most promising regions of the input space.

To create your own fitness function, define a function that takes an `Input` instance and returns a float value. The return value represents the 'fitness' of the given input, with higher values indicating better fitness. Here is a simple template:

```python
from evogfuzz.input import Input

def fitness_function_XYZ(inp: Input) -> float:
    # Implement your fitness function here.
    return 0.0
```

For instance, suppose we're interested in inputs that invoke the cosine function in our calculator. We could define a fitness function `fitness_function_cos` that assigns a high fitness value to inputs containing 'cos'. (**Note that this might not be the best fitness function to find new expcetions.**)

In [29]:
from difflib import SequenceMatcher
from evogfuzz.input import Input

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def fitness_function_XYZ(inp: Input) -> float:
    return similar("sqrt(-", str(inp)[:6])

In [30]:
"sqrt(-"[:6]

'sqrt(-'

In [31]:
from evogfuzz.input import Input

def fitness_function_cos(inp: Input) -> float:
    if 'cos' in str(inp):
        return 1.0
    else:
        return 0.0

Once your fitness function is defined, you can incorporate it into the `EvoGFuzz` instance by passing it as the `fitness_function` argument. 

In [34]:
epp = EvoGFuzz(
    grammar=CALCGRAMMAR,
    oracle=oracle,
    inputs=initial_inputs,
    fitness_function=fitness_function_XYZ,
    iterations=10,
)

found_exception_inputs = epp.fuzz()

print(f"EvoGFuzz found {len(found_exception_inputs)} bug-triggering inputs!")

for inp in found_exception_inputs:
    print(str(inp))

EvoGFuzz found 159 bug-triggering inputs!
sqrt(-9686)
sqrt(-81)
sqrt(-1)
sqrt(-676)
sqrt(-56)
sqrt(-675796)
sqrt(-865676)
sqrt(-119)
sqrt(-199)
sqrt(-7618)
sqrt(-149)
sqrt(-911119211119291)
sqrt(-1141141)
sqrt(-999449)
sqrt(-98566)
sqrt(-918)
sqrt(-919)
sqrt(-949)
sqrt(-5668)
sqrt(-68)
sqrt(-9999411)
sqrt(-967563)
sqrt(-699)
sqrt(-29)
sqrt(-165)
sqrt(-738)
sqrt(-31)
sqrt(-8816)
sqrt(-291)
sqrt(-149991)
sqrt(-669)
sqrt(-71)
sqrt(-11)
sqrt(-19)
sqrt(-429191144499)
sqrt(-9112)
sqrt(-49149)
sqrt(-6)
sqrt(-49491)
sqrt(-675)
sqrt(-111149)
sqrt(-39)
sqrt(-88)
sqrt(-585)
sqrt(-9944)
sqrt(-142914)
sqrt(-999)
sqrt(-96)
sqrt(-69)
sqrt(-697)
sqrt(-818)
sqrt(-89)
sqrt(-1191111991)
sqrt(-769)
sqrt(-8498)
sqrt(-611)
sqrt(-41)
sqrt(-149194994)
sqrt(-13)
sqrt(-57)
sqrt(-49)
sqrt(-144)
sqrt(-789)
sqrt(-941)
sqrt(-75671)
sqrt(-1191)
sqrt(-996)
sqrt(-9194)
sqrt(-8)
sqrt(-117)
sqrt(-41919)
sqrt(-86)
sqrt(-444191)
sqrt(-98894)
sqrt(-99)
sqrt(-121)
sqrt(-66)
sqrt(-65)
sqrt(-3)
sqrt(-67)
sqrt(-94999)
sqrt(-14

This way, the evolutionary grammar-based fuzzing process is now guided by your custom fitness function, focusing more on the areas you deem critical.

#### Evaluating Inputs Based on Custom Fitness Function

When utilizing a custom fitness function, such as `fitness_function_cos` in our case, we expect inputs containing 'cos' to achieve the highest fitness scores. This is because our fitness function assigns a score of 1.0 to any input that includes 'cos'.

To confirm this behavior, we retrieve all inputs generated during the fuzzing process using the `get_all_inputs()` method and sort these inputs based on their fitness scores.

In [35]:
all_generated_inputs = epp.get_all_inputs()
all_generated_inputs_sorted = sorted(all_generated_inputs, key=lambda inp: inp.fitness, reverse=True)

Let's display these sorted inputs along with their fitness scores. The inputs that contain 'cos' should appear first, demonstrating their high fitness value.

In [36]:
# investigate only the first 20 bug-triggering inputs
for inp in all_generated_inputs_sorted[:20]:
    print(f"{str(inp).ljust(40)} fitness: {inp.fitness}")

sqrt(-81)                                fitness: 1.0
sqrt(-98566)                             fitness: 1.0
sqrt(-9999411)                           fitness: 1.0
sqrt(-165)                               fitness: 1.0
sqrt(-8816)                              fitness: 1.0
sqrt(-149991)                            fitness: 1.0
sqrt(-111149)                            fitness: 1.0
sqrt(-69)                                fitness: 1.0
sqrt(-89)                                fitness: 1.0
sqrt(-1191111991)                        fitness: 1.0
sqrt(-769)                               fitness: 1.0
sqrt(-13)                                fitness: 1.0
sqrt(-144)                               fitness: 1.0
sqrt(-789)                               fitness: 1.0
sqrt(-941)                               fitness: 1.0
sqrt(-86)                                fitness: 1.0
sqrt(-99)                                fitness: 1.0
sqrt(-94999)                             fitness: 1.0
sqrt(-1494)                 

The resulting output validates the effectiveness of our custom fitness function. It shows how we can guide the evolutionary grammar-based fuzzing process towards specific regions of the input space, thereby facilitating targeted exploration and bug discovery.